In [7]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math
import os
from moviepy.editor import VideoFileClip
from IPython import display
import glob
import pickle
import ipywidgets
import math
import json
%matplotlib inline

In [36]:
class Camera:
    def __init__(self):
        self.calibrated = False
        
    def calibrate(self, calibration_images, show_annoted = False, show_corrected = False):
        w=9
        h=6
        pattern_size = (w,h)
        pattern_square_meters = 1


        # termination criteria
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

        # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
        single_object_points = np.zeros((w*h,3), np.float32)
        single_object_points[:,:2] = np.mgrid[0:w,0:h].T.reshape(-1,2) * pattern_square_meters

        all_object_points = []
        all_image_points = []
        im_shape = None

        for image_path in calibration_images:
            im = plt.imread(image_path)
            im_gray = cv2.cvtColor(im, cv2.COLOR_RGB2GRAY)
            im_gray = (im_gray*(np.max(im_gray)-np.min(im_gray))*255 ).astype('uint8')
            if im_shape == None:
                im_shape = im_gray.shape[::-1]
            if im_gray.shape[::-1] != im_shape:
                raise( ValueError('image different shape: '+image_path))
            found,corners = cv2.findChessboardCorners(im_gray,(9,6),None)
            if found==False: 
                continue
            corners2 = cv2.cornerSubPix(im_gray,corners,(11,11),(-1,-1),criteria)
            all_image_points.append(corners2)
            all_object_points.append(single_object_points)
            if show_annoted:
                plt.figure()
                im_annoted = cv2.drawChessboardCorners(im, pattern_size, corners2, found)
                plt.imshow(im_annoted)
        cx=im_shape[1]/2.
        cy=im_shape[0]/2.
        # set initial guess for camera matrix
        camera_matrix = np.array([ [1,0,cx], [0,1,cy], [0,0,1]])
        
        ret, camera_matrix, dist_coefs, rvecs, tvecs = cv2.calibrateCamera(
            all_object_points, 
            all_image_points, 
            im_shape, 
            camera_matrix,
            None,
            flags = cv2.CALIB_FIX_ASPECT_RATIO)
        
        self.dist_coefs = dist_coefs
        self.camera_matrix = camera_matrix
        self.calibrated = True
        if show_corrected:
            for image_path in calibration_images:
                im = plt.imread(image_path)   
                plt.figure()
                plt.imshow(cv2.undistort(im,mtx,dst))
    
    def save_calibration(self, path=None):
        if path == None:
            path = 'camera_cal.pickle'
        cal = {}
        cal['dist_coefs'] = self.dist_coefs
        cal['camera_matrix'] = self.camera_matrix
        with open( path, "wb" )  as f:
            pickle.dump(cal,f)

    def load_calibration(self, path=None):
        if path == None:
            path = 'camera_cal.pickle'
        cal = {}
        try:
            with open( path, "rb" )  as f:
                cal = pickle.load(f)
                self.dist_coefs = cal['dist_coefs']
                self.camera_matrix = cal['camera_matrix']
                return True
        except:
            pass
        return False
        
            
    
    def undistort(self, im):
        if not self.calibrated:
            raise(ValueError('camera must first be calibrated'))
        return cv2.undistort(im,self.camera_matrix,self.dist_coefs)

In [37]:
#run them all
chessboard_paths = glob.glob('chessboard_images/*')
all_calibrations = dict()
for chessboard_path in sorted(chessboard_paths):
    camera_name = chessboard_path.split('/')[1]
    camera = Camera()
    camera.calibrate(calibration_images = glob.glob(chessboard_path+'/*.png'))
    cal = dict()
    cal['dist_coefs'] = camera.dist_coefs.tolist()
    cal['camera_matrix'] = camera.camera_matrix.tolist()
    all_calibrations[camera_name] = cal
print(json.dumps(all_calibrations, indent=1))


{
 "elp1_right_320_240": {
  "camera_matrix": [
   [
    237.9216523346732,
    0.0,
    162.66964875313036
   ],
   [
    0.0,
    237.9216523346732,
    121.8319341355525
   ],
   [
    0.0,
    0.0,
    1.0
   ]
  ],
  "dist_coefs": [
   [
    -0.47070779623611164,
    0.30775419863475334,
    -0.001320709295522212,
    0.00022864252061203122,
    -0.11986176260540476
   ]
  ]
 },
 "elp1_right_640_480": {
  "camera_matrix": [
   [
    600.9862662720299,
    0.0,
    320.234407127221
   ],
   [
    0.0,
    600.9862662720299,
    242.0121323047996
   ],
   [
    0.0,
    0.0,
    1.0
   ]
  ],
  "dist_coefs": [
   [
    -0.4595948235549058,
    0.2782188398881187,
    0.0003603473845436643,
    0.000712435887913075,
    -0.10904216918995605
   ]
  ]
 },
 "elp1_left_800_600": {
  "camera_matrix": [
   [
    595.0485977008649,
    0.0,
    421.2771111599462
   ],
   [
    0.0,
    595.0485977008649,
    300.5649236438071
   ],
   [
    0.0,
    0.0,
    1.0
   ]
  ],
  "dist_coefs": [
